In [ ]:
# nltk
import nltk
nltk.download('stopwords')
import sqlite3, pandas as pd

def load_sql(db_name, tbl_name):
    """Load SQLite database."""
    con = sqlite3.connect(f"database/{db_name}.db")
    df = pd.read_sql(f"SELECT * FROM {tbl_name}", con)
    con.close()
    return df

In [ ]:
import numpy as np
import json, glob

# Gensim
import gensim, gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models

# spaCy
import spacy
from nltk.corpus import stopwords

# visualisation
import pyLDAvis

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
stopwords = stopwords.words('english')

In [ ]:
from nltk.tokenize import casual_tokenize
from collections import Counter

df = load_sql('tweets_v7', 'tweets_v7')

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # disable computationally expensive
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(df['text'])

In [ ]:
def gen_words(texts):
    final = []
    for text in texts:
        new = simple_preprocess(text, deacc=True)
        final.append(new)
    return final

data_words = gen_words(lemmatized_texts)

In [ ]:
# ----------------------------------------------------------------------------
#  BIGRAMS AND TRIGRAMS
# ----------------------------------------------------------------------------

In [ ]:
bigrams_phrases = gensim.models.Phrases(data_words, min_count=3, threshold=50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

In [ ]:
# ----------------------------------------------------------------------------
#  TF-IDF REMOVAL
# ----------------------------------------------------------------------------

In [76]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] # reintialise to be save
    tfidf_ids = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # the words with tfidf score 9 will be missing
    
    new bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]


In [ ]:
# id2word = corpora.Dictionary(data_words)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

In [83]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')

In [ ]:
# ----------------------------------------------------------------------------
#  VISUALISING DATA
# ----------------------------------------------------------------------------

In [84]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

/Users/ChrisP/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.332222  0.340916       1        1  20.110865
9      0.131713 -0.380148       2        1   8.595498
7     -0.322247 -0.229175       3        1   7.975173
0     -0.145926  0.349961       4        1   6.932144
5      0.246028 -0.211396       5        1   6.409608
12     0.331896 -0.072739       6        1   6.381766
4      0.046082  0.290163       7        1   4.765219
13    -0.140351 -0.251593       8        1   4.304870
1     -0.302359  0.124281       9        1   4.060780
19     0.210523  0.153317      10        1   4.024468
11     0.220033  0.014402      11        1   3.769332
17     0.027850  0.134123      12        1   3.734605
3     -0.267281 -0.034664      13        1   3.469307
10    -0.104730  0.152419      14        1   3.312756
2      0.009220 -0.237349      15        1   3.175509
18    -0.176999  0.077118      16        1   3.160205
8     -0.004627 -0.138636      17        1   2.647536
14    -0.112797 -0.054809      18        1   1.656211
15     0.047905 -0.022260      19        1   1.360025
16    -0.026153 -0.003932      20        1   0.154123, topic_info=          Term         Freq        Total Category  logprob  loglift
39         war  1371.000000  1371.000000  Default   30.000  30.0000
7      ukraine  1297.000000  1297.000000  Default   29.000  29.0000
53      people   330.000000   330.000000  Default   28.000  28.0000
313    missile   349.000000   349.000000  Default   27.000  27.0000
31          go   263.000000   263.000000  Default   26.000  26.0000
..         ...          ...          ...      ...      ...      ...
25       tweet     0.007575     3.278960  Topic20   -8.418   0.4047
26    irritate     0.007575     1.482718  Topic20   -8.418   1.1984
27    conflict     0.007575    70.367741  Topic20   -8.418  -2.6615
28   illegally     0.007575     1.482739  Topic20   -8.418   1.1984
29      invade     0.007575   142.600686  Topic20   -8.418  -3.3678

[767 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
295      17  0.902231   absolute
8         4  0.980158        act
464       5  0.882326     action
305       2  0.866346   aircraft
423       8  0.940666      allow
...     ...       ...        ...
58       10  0.989991      world
73        1  0.901178      worth
467       5  0.462442      wound
220       4  0.991982       year
446      10  0.923957  yesterday

[287 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 10, 8, 1, 6, 13, 5, 14, 2, 20, 12, 18, 4, 11, 3, 19, 9, 15, 16, 17])